In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段6：开始使用Vertex AI批量预测进行AutoML视频模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_video_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_video_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_video_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概述

本教程演示了如何从 `AutoML` 视频模型进行批量预测。

了解更多关于[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)。

### 目标

在本教程中，你将学习如何使用`Vertex AI Batch Prediction`与`AutoML`视频模型。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练一个`AutoML`模型。
- 使用JSONL输入进行批量预测。

使用批量预测和使用在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：针对整个实例集进行排队（批处理）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

数据集

本教程使用的数据集是来自麻省理工学院的[人类动作数据集](https://TODO)，该数据集的版本存储在一个公共云存储桶中。

费用

本教程使用 Google Cloud（GCP）的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 来根据您预期的使用情况生成一个费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，则您的环境已经满足运行此笔记本的所有要求。 您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。 您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。 以下步骤提供了一套简要的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。 激活虚拟环境。

4. 要安装Jupyter，请在终端shell中的命令行运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中的命令行运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

### 重新启动内核

安装额外包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

### GPU运行时

*如果有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置你的谷歌云项目

**无论你的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你首次创建一个帐户时，你会获得$300的免费信用以支付计算/存储成本。

2. [确保为你的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs、Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果你在本地运行此笔记本，你需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入你的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并会对以`$`为前缀的Python变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`命令获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个UUID，并将其附加到本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经过身份验证。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐号授予项目的访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”，选择**Vertex 管理员**。在筛选框中输入“存储对象管理员”，选择**存储对象管理员**。

点击创建。包含您密钥的 JSON 文件将下载到本地环境。

将服务帐号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入到下面的单元格中，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

在使用Python初始化Vertex AI SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在各个会话中保留的地方。

在下面设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化 Vertex AI SDK 用于 Python

为您的项目和相应的存储桶初始化 Vertex AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在您已准备好开始创建自己的AutoML视频分类模型。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

快速查看您的数据

您使用存储在公共云存储桶中的MIT Human Motion数据集的版本，使用一个CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标签架构。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="hmdb_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLVideoTrainingJob`类创建一个AutoML训练管道，其中包括以下参数：

- `display_name`：`TrainingJob`资源的可阅读名称。
- `prediction_type`：要为模型训练的任务类型。
  - `分类`：视频分类模型。
  - `目标跟踪`：视频目标跟踪模型。
  - `动作识别`：视频动作识别模型。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLVideoTrainingJob(
    display_name="hmdb_" + UUID, prediction_type="classification"
)

print(dag)

运行训练管道

接下来，通过调用方法`run`，使用以下参数来运行DAG以启动训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。

完成`run`方法后会返回`Model`资源。

训练管道的执行需要最多4小时。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="hmdb_" + UUID,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

批量预测简介

批量预测提供了对大量预测请求的离线批处理能力。资源只在批处理过程中进行分配，然后在批处理请求完成时取消分配。结果存储在云存储中，与在线预测不同，在线预测的结果作为HTTP响应包返回。

您的批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的web服务器必须支持JSONL格式，该web服务器将其转换为模型输入接口或服务功能接口直接支持的格式。对于批量预测，此JSONL格式称为`pivot`格式。

### 批量预测作业的输入格式

批处理服务器接受以下输入格式的AutoML视频模型：

- JSONL

### 批量预测作业的输出格式

批处理服务器接受以下输出格式的AutoML视频模型：

- JSONL


### Pivot格式

批处理服务器将输入格式转换为`pivot`（JSONL）格式如下：

**JSONL**

每个输入行（请求）应包含一个且仅有一个有效的json值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同格式生成pivot数据。生成的pivot数据然后被包装成一个载荷请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行的csv头将始终被忽略。字符串字段必须显式地用双引号引起，否则行将被丢弃并将输出解析错误消息到错误文件。非引用值总是作为浮点数传输。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每一行都被转换为JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器使用相同格式生成pivot数据。生成的pivot数据然后被包装成一个载荷请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块读取为二进制。然后将这些二进制对象序列化为ASCII字符串。预测服务器负责知道解码器以恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**FileList**

FileList格式包含一个文件列表。文件列表文件中的每一行指定一个单个文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器读取文件为二进制。二进制对象被序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

### 使用JSONL输入创建批量输入文件

现在创建一个批量输入文件，将其存储在您本地的云存储桶中。在这个例子中，您使用JSONL格式作为输入和输出。

### 获取测试项目

现在对您的Vertex AI模型进行批量预测。您可以从数据集中随意选择示例作为测试项目。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 5:
    _, test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 制作一个批处理输入文件

现在制作一个批处理输入文件，并将其存储在您的本地云存储桶中。批处理输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）的每一行制作一个字典条目。字典包含以下键/值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，是一个`avi`文件。
- `timeSegmentStart`：视频中要进行预测的开始时间戳。*注意*，时间戳必须以字符串形式指定，并以s（秒）、m（分钟）或h（小时）结尾。
- `timeSegmentEnd`：视频中要进行预测的结束时间戳。

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {
        "content": test_item_1,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")
    data = {
        "content": test_item_2,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 batch_predict() 方法来进行批量预测，需要提供以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `instances_format`：批量预测请求文件的格式："jsonl"、 "csv"、 "bigquery"、 "tf-record"、 "tf-record-gzip"或 "file-list"。
- `prediction_format`：批量预测响应文件的格式："jsonl"、 "csv"、 "bigquery"、 "tf-record"、 "tf-record-gzip"或 "file-list"。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为 True，则调用将阻塞，等待异步批量作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="hmdb_" + UUID,
    instances_format="jsonl",
    predictions_format="jsonl",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在 `batch_predict()` 方法中将参数 `sync` 设置为 `True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

###获取预测结果

接下来，从已完成的批量预测任务中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取每个云存储文件以及结果。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于0和1之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_batch_job = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)
if delete_batch_job:
    batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}